<a href="https://colab.research.google.com/github/LinaDanilina/practice/blob/master/practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
%cd drive/My Drive/instacart_2017_05_01

/content/drive/My Drive/instacart_2017_05_01


In [0]:
import pandas as pd
import operator
from collections import OrderedDict

In [0]:
df=pd.read_csv('data.csv',delimiter=';')

In [5]:
df

,Session id,Item id,time
0,1,a,0
1,1,b,1
2,1,c,2
3,1,f,3
4,2,c,4
5,2,d,5
6,3,b,6
7,3,c,7
8,3,d,8
9,4,c,9


In [0]:
groups=df.groupby('Session id')

In [7]:
seq=[]
names=[]
for name, group in groups:
  mydict = dict(zip(group['Item id'], group['time']))
  names.append(name)
  seq.append(mydict)
print(seq)
print(names)

[{'a': 0, 'b': 1, 'c': 2, 'f': 3}, {'c': 4, 'd': 5}, {'b': 6, 'c': 7, 'd': 8}, {'c': 9, 'e': 10}]
[1, 2, 3, 4]


In [0]:
for name, group in groups:
  d = {'Session id': names, '(Item id, interaction time)': seq}
map_s2i=pd.DataFrame(d)


In [9]:
map_s2i

,Session id,"(Item id, interaction time)"
0,1,"{'a': 0, 'b': 1, 'c': 2, 'f': 3}"
1,2,"{'c': 4, 'd': 5}"
2,3,"{'b': 6, 'c': 7, 'd': 8}"
3,4,"{'c': 9, 'e': 10}"


In [10]:
grouped=df.groupby('Item id')
seq_=[]
names_=[]
for name, group in grouped:
  mydict = dict(zip(group['Session id'], group['time']))
  names_.append(name)
  seq_.append(mydict)
print(seq_)
print(names_)

[{1: 0}, {1: 1, 3: 6}, {1: 2, 2: 4, 3: 7, 4: 9}, {2: 5, 3: 8}, {4: 10}, {1: 3}]
['a', 'b', 'c', 'd', 'e', 'f']


In [0]:
for name, group in groups:
  d = {'Item id': names_, '(Session id, interaction time)': seq_}
map_i2s=pd.DataFrame(d)

In [12]:
map_i2s

,Item id,"(Session id, interaction time)"
0,a,{1: 0}
1,b,"{1: 1, 3: 6}"
2,c,"{1: 2, 2: 4, 3: 7, 4: 9}"
3,d,"{2: 5, 3: 8}"
4,e,{4: 10}
5,f,{1: 3}


In [13]:
curr_ses={12:['d','a']}
for key in curr_ses:
  id_curr_ses=key

rec=[]
most_recent_sessions = {}
relevant_sessions=[]
#finding relevant sessions

for key in curr_ses:
  for item in curr_ses[key]:
    relevant_sessions.append(map_i2s[map_i2s['Item id'].str.contains(item)])
#selecting the most recent sessions
for ses in range(len(relevant_sessions)):
  max_time=0
  max_k=0
  for dic in relevant_sessions[ses]['(Session id, interaction time)']:
    for key in dic:
      if dic[key]>=max_time:
        max_time=dic[key]
        max_k=key
  most_recent_sessions[max_k]=max_time
  k_recent=len(most_recent_sessions)
print(most_recent_sessions)
print(k_recent)

{3: 8, 1: 0}
2


In [0]:
def sim(x,j):
  return 1

In [20]:
k_top=2 
NN={}
for key in most_recent_sessions:
  max_sim=0
  if sim(key, id_curr_ses)>max_sim:
    max_sim=sim(key, id_curr_ses)
    if k_top!=len(NN):   #chosing k_top the most similar sessions
      tmp=map_s2i[map_s2i['Session id']==key]['(Item id, interaction time)']
      for t in tmp:
        NN[key]=t
    
print(NN)

 

{3: {'b': 6, 'c': 7, 'd': 8}, 1: {'a': 0, 'b': 1, 'c': 2, 'f': 3}}


In [97]:
relevant_items=[]
NN_id=[]
for key in NN:
  NN_id.append(key)
  for item in NN[key]:
    relevant_items.append(item)
print(relevant_items)  
print(NN_id)

['b', 'c', 'd', 'a', 'b', 'c', 'f']
[3, 1]


In [103]:
it_sc={}
for item in relevant_items:
  score=0
  for ids in NN_id:
    for el in df[(df['Session id']==ids)]['Item id']:
      if item==el:
        score+=sim(id_curr_ses,ids)*1
      else:
        score+=sim(id_curr_ses,ids)*0
  it_sc[item]=score
print(it_sc)

{'b': 2, 'c': 2, 'd': 1, 'a': 1, 'f': 1}


In [0]:
N=2 #recommend N nearest items
sorted_pairs = sorted(((k, v) for k, v in it_sc.items()),
                     key=lambda pair: pair[1], reverse=True)
output = OrderedDict()
for k, v in sorted_pairs:
    if k not in output:
        output[k] = v
        if len(output) == N:
            break


In [130]:
print('Recommended items are:')
for item in output:
  print(item)


Recommended items are:
b
c
